<a href="https://colab.research.google.com/github/prakunjpratapsingh/movie-recommendation-UI/blob/main/AI_System_to_Digitize_Handwritten_Exams_in_German_Language_Studies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00


In [2]:
! pip install datasets

In [3]:
!pip install evaluate

In [4]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import evaluate
from torch.optim import AdamW
from tqdm.notebook import tqdm


In [5]:
# Load CER metric
cer_metric = evaluate.load("cer")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


**Dataset** **Description**

In [6]:
# Load dataset
dataset = load_dataset('fhswf/german_handwriting')
df = pd.DataFrame(dataset['train'])

README.md:   0%|          | 0.00/893 [00:00<?, ?B/s]

train-00000-of-00008.parquet:   0%|          | 0.00/366M [00:00<?, ?B/s]

train-00001-of-00008.parquet:   0%|          | 0.00/18.8M [00:00<?, ?B/s]

train-00002-of-00008.parquet:   0%|          | 0.00/55.5M [00:00<?, ?B/s]

train-00003-of-00008.parquet:   0%|          | 0.00/67.7M [00:00<?, ?B/s]

train-00004-of-00008.parquet:   0%|          | 0.00/165M [00:00<?, ?B/s]

train-00005-of-00008.parquet:   0%|          | 0.00/559M [00:00<?, ?B/s]

train-00006-of-00008.parquet:   0%|          | 0.00/184M [00:00<?, ?B/s]

train-00007-of-00008.parquet:   0%|          | 0.00/381M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10854 [00:00<?, ? examples/s]

In [7]:
df.head()

,image,text
0,<PIL.PngImagePlugin.PngImageFile image mode=RG...,- Terminvorschlag bis
1,<PIL.PngImagePlugin.PngImageFile image mode=RG...,- Module an Michael
2,<PIL.PngImagePlugin.PngImageFile image mode=RG...,Handschrift
3,<PIL.PngImagePlugin.PngImageFile image mode=RG...,Datenschutz bei
4,<PIL.PngImagePlugin.PngImageFile image mode=RG...,-> Urheberrecht


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10854 entries, 0 to 10853
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   10854 non-null  object
 1   text    10844 non-null  object
dtypes: object(2)
memory usage: 169.7+ KB


In [9]:
df.isnull().sum()

,0
image,0
text,10


In [10]:
missing_text_rows = df[df['text'].isnull()]
print("Rows with missing 'text' values:")
print(missing_text_rows)

Rows with missing 'text' values:
                                                   image  text
3060   <PIL.PngImagePlugin.PngImageFile image mode=RG...  None
4436   <PIL.PngImagePlugin.PngImageFile image mode=RG...  None
4536   <PIL.PngImagePlugin.PngImageFile image mode=RG...  None
6265   <PIL.PngImagePlugin.PngImageFile image mode=RG...  None
6540   <PIL.PngImagePlugin.PngImageFile image mode=RG...  None
7391   <PIL.PngImagePlugin.PngImageFile image mode=RG...  None
7944   <PIL.PngImagePlugin.PngImageFile image mode=RG...  None
9531   <PIL.PngImagePlugin.PngImageFile image mode=RG...  None
10206  <PIL.PngImagePlugin.PngImageFile image mode=RG...  None
10333  <PIL.PngImagePlugin.PngImageFile image mode=RG...  None


In [11]:
df = df.dropna(subset=['text'])

In [12]:
df.isnull().sum()

,0
image,0
text,0


In [13]:
class IAMDataset(Dataset):
    def __init__(self, df, processor, max_target_length=128):
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = self.df['image'][idx]  # PIL.Image object
        text = self.df['text'][idx]

        # Process image
        processed_image = self.processor(image, return_tensors="pt")
        pixel_values = processed_image.pixel_values.squeeze(0)

        # Tokenize text
        labels = self.processor.tokenizer(
            text,
            padding="max_length",
            max_length=self.max_target_length,
            truncation=True,
        ).input_ids
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        return {"pixel_values": pixel_values, "labels": torch.tensor(labels)}

In [14]:
# Split dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
cleaned_train_df = train_df[train_df['text'].notnull() & (train_df['text'] != "")]
cleaned_train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [15]:
# Validate and fix images
def validate_images(df):
    for idx, image in enumerate(df['image']):
        try:
            if not isinstance(image, Image.Image):
                raise ValueError(f"Image at index {idx} is not a valid PIL image.")
            if image.mode != "RGB":
                df['image'][idx] = image.convert("RGB")
        except Exception as e:
            print(f"Error at index {idx}: {e}")

validate_images(cleaned_train_df)
validate_images(test_df)

<ipython-input-15-ea72c002e636>:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['image'][idx] = image.convert("RGB")


In [16]:
# Instantiate the processor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")

preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [17]:
# Define datasets
train_dataset = IAMDataset(df=cleaned_train_df, processor=processor)
eval_dataset = IAMDataset(df=test_df, processor=processor)


In [18]:
# Collate function
def collate_fn(batch):
    pixel_values = torch.stack([item["pixel_values"] for item in batch])
    labels = torch.nn.utils.rnn.pad_sequence(
        [item["labels"] for item in batch], batch_first=True, padding_value=-100
    )
    return {"pixel_values": pixel_values, "labels": labels}

In [19]:
# Data loaders
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn
)
eval_dataloader = torch.utils.data.DataLoader(
    eval_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn
)

**Model Definition & Fine-tuning**

In [20]:
# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")
model.to(device)

config.json:   0%|          | 0.00/4.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.48.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linea

In [21]:
# Configure model
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [22]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

**Training and Evaluation Code**

In [23]:
# Training Loop
for epoch in range(5):
    model.train()  # Set the model to training mode
    train_loss = 0.0  # Initialize training loss accumulator

    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}"):
        # Move batch data to the device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(pixel_values=batch["pixel_values"], labels=batch["labels"])
        loss = outputs.loss

        # Backward pass to calculate gradients
        loss.backward()

        # Step optimizer and zero gradients
        optimizer.step()
        optimizer.zero_grad()

        # Accumulate training loss
        train_loss += loss.item()

    # Print average training loss after each epoch
    print(f"Loss after epoch {epoch}: {train_loss / len(train_dataloader)}")


Training Epoch 1:   0%|          | 0/1085 [00:00<?, ?it/s]

Loss after epoch 0: 2.5130005232200094


Training Epoch 2:   0%|          | 0/1085 [00:00<?, ?it/s]

Loss after epoch 1: 1.5996478707010295


Training Epoch 3:   0%|          | 0/1085 [00:00<?, ?it/s]

Loss after epoch 2: 1.0869664058981952


Training Epoch 4:   0%|          | 0/1085 [00:00<?, ?it/s]

Loss after epoch 3: 0.9348804777668368


Training Epoch 5:   0%|          | 0/1085 [00:00<?, ?it/s]

Loss after epoch 4: 0.8751201468129312


In [24]:
# Enhanced Evaluation Loop with Correct Batch Skipping
model.eval()  # Set the model to evaluation mode
valid_cer = 0.0  # Initialize CER accumulator
problematic_batches = []  # List to store problematic batches

with torch.no_grad():  # Disable gradient computation during evaluation
    for i, batch in enumerate(tqdm(eval_dataloader, desc="Evaluating")):
        try:
            # Skip specific batches 185 and 186
            if i + 1 in [185, 186]:  # Adjust for 1-based indexing in descriptions
                print(f"Skipping batch {i + 1}")
                problematic_batches.append(batch)  # Save skipped batch for debugging
                continue

            # Move batch data to the device
            batch = {k: v.to(device) for k, v in batch.items()}

            # Debugging: Check batch content
            print(f"\nProcessing batch {i + 1}")
            print(f"Batch keys: {batch.keys()}")
            print(f"Batch pixel_values shape: {batch['pixel_values'].shape}")
            print(f"Batch labels shape: {batch['labels'].shape}")

            # Ensure batch contains valid data
            if batch["pixel_values"].nelement() == 0 or batch["labels"].nelement() == 0:
                raise ValueError(f"Empty data in batch {i + 1}")

            # Generate predictions
            outputs = model.generate(pixel_values=batch["pixel_values"])
            print(f"Generated outputs shape: {outputs.shape}")

            pred_texts = processor.batch_decode(outputs, skip_special_tokens=True)
            print(f"Predicted texts: {pred_texts[:3]}")  # Show first 3 predictions

            # Process labels for comparison
            label_ids = batch["labels"].clone()
            label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
            label_texts = processor.batch_decode(label_ids, skip_special_tokens=True)
            print(f"Label texts: {label_texts[:3]}")  # Show first 3 labels

            # Compute CER
            cer = cer_metric.compute(predictions=pred_texts, references=label_texts)
            print(f"Current CER: {cer}")
            valid_cer += cer

        except Exception as e:
            print(f"Error in batch {i + 1}: {e}")
            problematic_batches.append(batch)  # Save problematic batch
            continue  # Skip the problematic batch and proceed

# Final CER calculation
num_skipped_batches = len(problematic_batches)
if valid_cer > 0:
    avg_cer = valid_cer / (len(eval_dataloader) - num_skipped_batches)  # Exclude skipped batches
    print(f"Validation CER after evaluation: {avg_cer:.4f}")
else:
    print("No valid CER calculated due to errors.")



Evaluating:   0%|          | 0/272 [00:00<?, ?it/s]


Processing batch 1
Batch keys: dict_keys(['pixel_values', 'labels'])
Batch pixel_values shape: torch.Size([8, 3, 384, 384])
Batch labels shape: torch.Size([8, 128])


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1528: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Generated outputs shape: torch.Size([8, 24])
Predicted texts: ['- Rückverschlingung aller Staaten', '4. Die ClasssIM Modell inhalisieren', 'GETT & POSst']
Label texts: ['- Gleichberechtigung aller Staaten', '#OneClassSVM Modell initialisieren', 'GET & POST']
Current CER: 0.17216117216117216

Processing batch 2
Batch keys: dict_keys(['pixel_values', 'labels'])
Batch pixel_values shape: torch.Size([8, 3, 384, 384])
Batch labels shape: torch.Size([8, 128])
Generated outputs shape: torch.Size([8, 25])
Predicted texts: ['Datenbanken speichern Informationen.', 'In der Pränkal der Bundesspunkt wird', '- sünde unterte und Teilweise Entlassungen ab QOA']
Label texts: ['Datenbanken speichern Informationen.', 'In der Präambel der Bundesrepublik wird', '- Sinkende Urteile und Teilweise Entlassungen ab 1949']
Current CER: 0.14868804664723032

Processing batch 3
Batch keys: dict_keys(['pixel_values', 'labels'])
Batch pixel_values shape: torch.Size([8, 3, 384, 384])
Batch labels shape: torch.Size([8,

**Testing Code**

In [ ]:
import torch

# Set the model to evaluation mode
model.eval()

# Function to test a single input line
def test_model_on_line(input_text, processor, model, device):
    try:
        # Preprocess the input text
        inputs = processor(text=input_text, return_tensors="pt", padding=True, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}  # Move to device

        # Generate predictions
        with torch.no_grad():
            outputs = model.generate(pixel_values=inputs["pixel_values"])

        # Decode the predicted text
        predicted_text = processor.batch_decode(outputs, skip_special_tokens=True)[0]

        print("\n===== Model Prediction =====")
        print(f"Input Text: {input_text}")
        print(f"Predicted Text: {predicted_text}\n")

        return predicted_text
    except Exception as e:
        print(f"Error during testing: {e}")
        return None

# Example usage:
input_text = "Your sample input line here"  # Replace with the actual input
predicted_output = test_model_on_line(input_text, processor, model, device)

**Extracting Text From Image**

In [ ]:
import easyocr
reader = easyocr.Reader(['de'])
translator = Translator()

In [ ]:
import PIL
from PIL import ImageDraw
im = PIL.Image.open("/content/Screenshot 2025-01-07 215746.png")

In [ ]:
bounds = reader.readtext('/content/Screenshot 2025-01-07 215746.png', add_margin=0.55, width_ths=0.7, link_threshold=0.8, decoder='beamsearch',blocklist='=-')
bounds

In [ ]:
def draw_boxes(image, bounds, color='yellow', width=2):
    draw = ImageDraw.Draw(image)
    for bound in bounds:
        p0, p1, p2, p3 = bound[0]
        draw.line([*p0, *p1, *p2, *p3, *p0], fill=color, width=width)
    return image

draw_boxes(im, bounds)

In [ ]:
text_list = reader.readtext('/content/Screenshot 2025-01-07 215746.png', add_margin=0.55, width_ths=0.7, link_threshold=0.8, decoder='beamsearch',blocklist='=-', detail=0)
text_list

In [ ]:
text_comb=' '.join(text_list)
text_comb